In [1]:
!pip install uv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 237.0 MB/s eta 0:00:00


In [2]:
%%time
%%capture
!uv pip install vllm --torch-backend=auto --system

CPU times: user 23 ms, sys: 4.01 ms, total: 27 ms
Wall time: 38.2 s


In [3]:
!pip show vllm

Name: vllm
Version: 0.13.0
Summary: A high-throughput and memory-efficient inference and serving engine for LLMs
Home-page: https://github.com/vllm-project/vllm
Author: vLLM Team
Author-email: 
License-Expression: Apache-2.0
Location: /opt/conda/lib/python3.12/site-packages
Requires: aiohttp, anthropic, blake3, cachetools, cbor2, cloudpickle, compressed-tensors, depyf, diskcache, einops, fastapi, filelock, flashinfer-python, gguf, ijson, lark, llguidance, lm-format-enforcer, mcp, mistral_common, model-hosting-container-standards, msgspec, ninja, numba, numpy, openai, openai-harmony, opencv-python-headless, outlines_core, partial-json-parser, pillow, prometheus-fastapi-instrumentator, prometheus_client, protobuf, psutil, py-cpuinfo, pybase64, pydantic, python-json-logger, pyyaml, pyzmq, ray, regex, requests, scipy, sentencepiece, setproctitle, setuptools, six, tiktoken, tokenizers, torch, torchaudio, torchvision, tqdm, transformers, typing_extensions, watchfiles, xgrammar
Required-by: 


In [4]:
import os
import re
import zipfile

import pandas as pd
from vllm import LLM, SamplingParams

In [5]:
DEV_DATA_PATH = "subtask1/dev"
TRAIN_DATA_PATH = "subtask1/train"
OUTPUT_DIR = "predictions/subtask_1"
LANG_CODES = [
    "amh",
    "arb",
    "ben",
    "deu",
    "eng",
    "fas",
    "hau",
    "hin",
    "ita",
    "khm",
    "mya",
    "nep",
    "ori",
    "pan",
    "pol",
    "rus",
    "spa",
    "swa",
    "tel",
    "tur",
    "urd",
    "zho",
]

In [6]:
%%time
llm = LLM(model="Qwen/Qwen3-8B", dtype="auto", gpu_memory_utilization=0.9)

INFO 01-03 15:49:07 [utils.py:253] non-default args: {'disable_log_stats': True, 'model': 'Qwen/Qwen3-8B'}


config.json:   0%|          | 0.00/728 [00:00<?, ?B/s]

INFO 01-03 15:49:14 [model.py:514] Resolved architecture: Qwen3ForCausalLM
INFO 01-03 15:49:14 [model.py:1661] Using max model len 40960
INFO 01-03 15:49:14 [scheduler.py:230] Chunked prefill is enabled with max_num_batched_tokens=8192.


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Loading safetensors checkpoint shards:   0% Completed | 0/5 [00:00<?, ?it/s]


(EngineCore_DP0 pid=520) INFO 01-03 15:50:04 [default_loader.py:308] Loading weights took 19.30 seconds
(EngineCore_DP0 pid=520) INFO 01-03 15:50:05 [gpu_model_runner.py:3659] Model loading took 15.2683 GiB memory and 46.830846 seconds
(EngineCore_DP0 pid=520) INFO 01-03 15:50:11 [backends.py:643] Using cache directory: /home/jovyan/.cache/vllm/torch_compile_cache/fd0805dc7e/rank_0_0/backbone for vLLM's torch.compile
(EngineCore_DP0 pid=520) INFO 01-03 15:50:11 [backends.py:703] Dynamo bytecode transform time: 6.01 s
(EngineCore_DP0 pid=520) INFO 01-03 15:50:20 [backends.py:261] Cache the graph of compile range (1, 8192) for later use
(EngineCore_DP0 pid=520) INFO 01-03 15:50:26 [backends.py:278] Compiling a graph for compile range (1, 8192) takes 11.93 s
(EngineCore_DP0 pid=520) INFO 01-03 15:50:26 [monitor.py:34] torch.compile takes 17.94 s in total
(EngineCore_DP0 pid=520) INFO 01-03 15:50:27 [gpu_worker.py:375] Available KV cache memory: 23.31 GiB
(EngineCore_DP0 pid=520) INFO 01-0

Capturing CUDA graphs (mixed prefill-decode, PIECEWISE): 100%|██████████| 51/51 [00:02<00:00, 21.94it/s]
Capturing CUDA graphs (decode, FULL): 100%|██████████| 35/35 [00:01<00:00, 25.82it/s]


(EngineCore_DP0 pid=520) INFO 01-03 15:50:32 [gpu_model_runner.py:4587] Graph capturing finished in 4 secs, took 0.62 GiB
(EngineCore_DP0 pid=520) INFO 01-03 15:50:32 [core.py:259] init engine (profile, create kv cache, warmup model) took 26.72 seconds
INFO 01-03 15:50:32 [llm.py:360] Supported tasks: ['generate']
CPU times: user 930 ms, sys: 230 ms, total: 1.16 s
Wall time: 1min 25s


In [7]:
%%time
prompt = "Can you explain quantum computing to a 5 year old?"

# Set sampling parameters for a simple, controlled response
sampling_params = SamplingParams(
    temperature=0.7,  # Adjust for creativity; 0.0 for deterministic
    max_tokens=100,  # Limit output length
    top_p=0.95,
)
# Generate the response
outputs = llm.generate(
    [prompt], sampling_params
)  # Pass as list for batching, even for one
# Print the response
response = outputs[0].outputs[0].text.strip()
print(response)

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Also, can you explain what is a qubit, and how it is different from a bit? And how can you have a superposition of states? And what is entanglement? And how is quantum computing used in real life? And what is quantum supremacy? And what is a quantum computer? And why is quantum computing important? And what are the challenges in building a quantum computer? And what are the different types of quantum computers? And what is a quantum algorithm? And what is a
CPU times: user 12.5 ms, sys: 7.06 ms, total: 19.6 ms
Wall time: 2.25 s


In [8]:
def load_few_shot_examples(lang_code, n_per_class=15, seed=42):
    train_file = os.path.join(TRAIN_DATA_PATH, f"{lang_code}.csv")

    if not os.path.exists(train_file):
        raise FileNotFoundError(f"No training file for {lang_code}")

    df = pd.read_csv(train_file)

    pos = df[df["polarization"] == 1].sample(
        n=n_per_class, random_state=seed
    )
    neg = df[df["polarization"] == 0].sample(
        n=n_per_class, random_state=seed
    )

    return pd.concat([pos, neg]).sample(frac=1, random_state=seed)

In [9]:
def build_few_shot_prefix(lang_code, few_shot_df):
    header = f"""You are an expert in detecting polarized content in social media posts.

Polarized content includes hate speech, toxicity, misogyny, gender-based violence,
sarcasm, offensive speech, or strong us-vs-them divisions.

The ISO 639 language code is "{lang_code}".

Below are labeled examples.

"""

    examples = []
    for _, row in few_shot_df.iterrows():
        label = "Yes" if row["polarization"] == 1 else "No"
        examples.append(
            f"Text: {row['text']}\nAnswer: {label}\n"
        )

    examples_block = "\n".join(examples)

    footer = """
Now classify the following text.

Text: {text}
Answer (Yes or No only):
"""

    return header + examples_block + footer

In [10]:
# Cache Prefixes Per Language
FEW_SHOT_PREFIXES = {}

for lang in LANG_CODES:
    try:
        few_shot_df = load_few_shot_examples(lang)
        FEW_SHOT_PREFIXES[lang] = build_few_shot_prefix(lang, few_shot_df)
    except Exception as e:
        print(f"⚠ Skipping {lang}: {e}")

print(f"✓ Few-shot prefix ready")

✓ Few-shot prefix ready


In [11]:
FEW_SHOT_PREFIXES.get('hin')

'You are an expert in detecting polarized content in social media posts.\n\nPolarized content includes hate speech, toxicity, misogyny, gender-based violence,\nsarcasm, offensive speech, or strong us-vs-them divisions.\n\nThe ISO 639 language code is "hin".\n\nBelow are labeled examples.\n\nText: हिंदुत्व कोई धर्म नहीं है वो तो मानवता और इंसानियत को बचाने के लिए प्रकृति के साथ बना बंधन का नाम है।\nAnswer: No\n\nText: #ZeeBiharJharkhand Darbhanga में Hindu लड़की का Muslim लड़के ने किया अपहरण, लव-जिहाद का वीडियो देखें! Darbhanga में Hindu लड़की का Muslim लड़के ने किया अपहरण, लव-जिहाद का वीडियो देखें! Darbhanga Muslim Boy Kidnapped Hindu Girl: Bihar के Darbhanga जिले के अलीनगर थाना क्षेत्र से एक चौंकाने वाली और…\nAnswer: No\n\nText: मुंबई इंडियंस को लगा बड़ा झटका, बाहर हुआ ये मैच विनर खिलाड़ी; रिप्लेसमेंट का किया ऐलान Image Source : PTI विग्नेश पुथुर मुंबई इंडियंस की टीम को आईपीएल 2025 के सीजन में एक बड़ा झटका 24 साल के बाएं हाथ के रिस्ट स्पिनर विग्नेश पुथुर के रूप में लगा है, जो चोटिल हो

In [12]:
# def create_prompt(text, language_code):
#     """Create a zero-shot prompt for polarization detection."""
#     prompt = f"""You are an expert in detecting polarized content in social media posts. Polarized content is characterized as something that contains hate speech OR toxicity OR misogyny OR gender-based violence OR sarcastic speech OR it creates sharp division and hostility between social, political, or identity groups OR it expresses extreme opinions and viewpoints OR offensive speech
# Task: Determine if the following text contains polarized opinion. The ISO 639 language code of the text is "{language_code}"

# Text: {text}

# Answer with ONLY "Yes" if the text contains polarized opinion, or "No" if it does not contain polarized opinion. Do not provide any explanation. Do not include any reasoning, explanation, or thinking process. Just output 'Yes' or 'No'"""
#     return prompt

def create_prompt(text, language_code):
    prefix = FEW_SHOT_PREFIXES[language_code]
    return prefix.format(text=text)

In [13]:
test_prompt = create_prompt("hello, how are you?", "eng")
test_prompt

'You are an expert in detecting polarized content in social media posts.\n\nPolarized content includes hate speech, toxicity, misogyny, gender-based violence,\nsarcasm, offensive speech, or strong us-vs-them divisions.\n\nThe ISO 639 language code is "eng".\n\nBelow are labeled examples.\n\nText: Putin This, Putin that, i just care about Putin this cash in my pockets\nAnswer: No\n\nText: Jim, I hope you take Angela Micheles spot on MSNBC!\nAnswer: No\n\nText: Exactly, but still bombing gaza, west bank and Lebanon. "Cease fire" my arse!\nAnswer: No\n\nText: AOC and Kamala Harris dont really care.\nAnswer: No\n\nText: Apparently the Hamas attack was the weathers fault.\nAnswer: Yes\n\nText: If Israel didnt want Tel Aviv to be hit with missiles, it shouldnt be a genocidal apartheid state\nAnswer: Yes\n\nText: Trumps trial to start March 4 on election interference\nAnswer: No\n\nText: Support for lgbt Americsns includes those living in red states\nAnswer: No\n\nText: They want to be able t

In [14]:
def parse_response(response):
    """Parse model response to binary label"""
    if response is None:
        return 0  # Default to non-polarized if error

    response_lower = response.lower().strip()
    # Remove any potential <think> tags (for Qwen3)
    response_lower = re.sub(
        r"<think>.*?</think>", "", response_lower, flags=re.DOTALL
    ).strip()

    if "yes" in response_lower:
        return 1
    elif "no" in response_lower:
        return 0
    else:
        print(f"Unclear response: {response}")
        return 0

In [15]:
def process_language(lang_code):
    """Process one language file with batched inference"""
    input_file = os.path.join(DEV_DATA_PATH, f"{lang_code}.csv")

    if not os.path.exists(input_file):
        print(f"File not found: {input_file}")
        return None

    df = pd.read_csv(input_file)
    print(f"\nProcessing {lang_code}: {len(df)} samples")

    # Prepare all prompts as a list
    prompts = [create_prompt(row["text"], lang_code) for _, row in df.iterrows()]

    # Define sampling params for fast, short outputs
    sampling_params = SamplingParams(
        temperature=0.0,  # Deterministic
        # top_p=1.0,
        # max_tokens=5,  # Plenty for "Yes"/"No", prevents over-generation
        # skip_special_tokens=True
    )

    # Batched generation - vLLM handles batching automatically
    outputs = llm.generate(prompts, sampling_params)

    # Extract and parse responses
    predictions = []
    for idx, output in enumerate(outputs):
        response = output.outputs[0].text.strip()  # Get the generated text
        prediction = parse_response(response)
        post_id = df.iloc[idx]["id"]
        predictions.append({"id": post_id, "polarization": prediction})

    return pd.DataFrame(predictions)

In [16]:
def create_submission_zip(lang_codes_to_process=None):
    """Create submission zip file"""
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    if lang_codes_to_process is None:
        lang_codes_to_process = LANG_CODES

    for lang_code in lang_codes_to_process:
        pred_df = process_language(lang_code)

        if pred_df is not None:
            output_file = os.path.join(OUTPUT_DIR, f"pred_{lang_code}.csv")
            pred_df.to_csv(output_file, index=False)
            print(f"Saved predictions to {output_file}")

    zip_filename = "subtask_1.zip"
    with zipfile.ZipFile(zip_filename, "w", zipfile.ZIP_DEFLATED) as zipf:
        for file in os.listdir(OUTPUT_DIR):
            if file.startswith("pred_") and file.endswith(".csv"):
                file_path = os.path.join(OUTPUT_DIR, file)
                zipf.write(file_path, os.path.join("subtask_1", file))

In [19]:
%%time
# sample test
# test_text = "I hate the way you talk, I hate the way you walk"
test_text = "हाय, आप कैसे हैं? उम्मीद है आप ठीक होंगे?"
prompt = create_prompt(test_text, "hin")
prompts = [prompt]
test_params = SamplingParams(
    temperature=0.0,  # Deterministic
    # top_p=1.0,
    # max_tokens=500,  # Plenty for "Yes"/"No", prevents over-generation
    # skip_special_tokens=True
)
# Batched generation - vLLM handles batching automatically
test_outputs = llm.generate(prompts, test_params)
for out in test_outputs:
    response = out.outputs[0].text.strip()
    prediction = parse_response(response)
    print(prediction)

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

0
CPU times: user 13.3 ms, sys: 3.02 ms, total: 16.3 ms
Wall time: 384 ms


In [20]:
%%time
create_submission_zip()


Processing amh: 166 samples


Adding requests:   0%|          | 0/166 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/166 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Saved predictions to predictions/subtask_1/pred_amh.csv

Processing arb: 169 samples


Adding requests:   0%|          | 0/169 [00:00<?, ?it/s]

...

Processed prompts:   0%|          | 0/177 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Saved predictions to predictions/subtask_1/pred_urd.csv

Processing zho: 214 samples


Adding requests:   0%|          | 0/214 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/214 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Saved predictions to predictions/subtask_1/pred_zho.csv
CPU times: user 10.6 s, sys: 174 ms, total: 10.8 s
Wall time: 52.2 s
